In [1]:
! git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214 (from 1)
Receiving objects: 100% (14214/14214), 239.65 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [2]:
import csv
data = {}
with open ('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader) #pointing to the secon row, skipp the first row that is header
  for row in reader:
    key = row[2].lower()  # Extract the third column (index 2) and convert it to lowercase to use as the key.
    if key in data:   # Check if the key already exists in the dictionary.
      data[key].append(row[1])  # If the key exists, append the second column (index 1) value to the existing list.
    else:
      data[key] = [row[1]]   # If the key doesn't exist, create a new list with the second column (index 1) value.


In [4]:
 emotion_list = list(data.keys())  # Convert the dictionary keys (emotions) to a list.
 emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [6]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [7]:
from shutil import copyfile  # Import the copyfile function from the shutil module.

split_size = 0.8  # Define the split size for dividing data into training and testing sets (80% for training).

for emotion, images in data.items():  # Iterate over each emotion and its corresponding list of images in the data dictionary.
  train_size = int(split_size*len(images))  # Calculate the number of images for the training set based on the split size.
  train_images = images[:train_size]  # Extract the training images from the list.
  test_images = images[train_size:]  # Extract the testing images from the list.
  for image in train_images:  # Iterate over each image in the training set.
    source = os.path.join('/content/facial_expressions/images', image)  # Construct the source path for the image.
    dest = os.path.join('/content/master_data/training', emotion, image)  # Construct the destination path for the image in the training directory.
    copyfile(source, dest)  # Copy the image from the source to the destination.
  for image in test_images:  # Iterate over each image in the testing set.
    source = os.path.join('/content/facial_expressions/images', image)  # Construct the source path for the image.
    dest = os.path.join('/content/master_data/testing', emotion, image)  # Construct the destination path for the image in the testing directory.
    copyfile(source, dest)  # Copy the image from the source to the destination.

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [17]:
model = tf.keras.models.Sequential([
    # First convolutional layer with 16 filters, 3x3 kernel, ReLU activation, and input shape of (100, 100, 3)
    Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
    # Max pooling layer with pool size of 2x2
    MaxPooling2D(2, 2),
    # Second convolutional layer with 32 filters, 3x3 kernel, and ReLU activation
    Conv2D(32, (3,3), activation='relu'),
    # Max pooling layer with pool size of 2x2
    MaxPooling2D(2, 2),
    # Third convolutional layer with 64 filters, 3x3 kernel, and ReLU activation
    Conv2D(64, (3,3), activation='relu'),
    # Max pooling layer with pool size of 2x2
    MaxPooling2D(2, 2),
    # Flatten layer to convert the multi-dimensional output to a 1D array
    Flatten(),
    # Dense layer with 1024 units and ReLU activation
    Dense(1024, activation='relu'),
    # Output layer with 8 units (for 8 emotions) and softmax activation for probability distribution
    Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 98, 98, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 49, 49, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 47, 47, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │       6,554,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │           8,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,586,408 (25.13 MB)

 Trainable params: 6,586,408 (25.13 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [16]:
# Define the EarlyStopping callback
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

# Use model.fit instead of model.fit_generator
model.fit(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 22s 214ms/step - acc: 0.4457 - loss: 3.7021 - val_acc: 0.5011 - val_loss: 1.0418
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 162ms/step - acc: 0.4994 - loss: 1.0439 - val_acc: 0.5011 - val_loss: 1.0435
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 23s 197ms/step - acc: 0.4921 - loss: 1.0423 - val_acc: 0.5011 - val_loss: 1.0457


Based on these results, we can infer that the model is struggling to learn effectively and is possibly overfitting to the training data.

To improve accuracy, we can try adding more layers, applying regularization techniques like Dropout or BatchNormalization, and augmenting the dataset using ImageDataGenerator.

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing - Image Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,
    zoom_range=0.2,  # Randomly zoom in on images
    horizontal_flip=True,  # Randomly flip images
    fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_dir,  # Update the path
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical')

# Define Model Architecture
model = Sequential()

# Add Convolutional Layers with Batch Normalization and Dropout
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Dropout to reduce overfitting

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten the layers and add Fully Connected Layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Dropout to reduce overfitting

# Output Layer
model.add(Dense(8, activation='softmax'))  # 8 output classes for facial expressions

# Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the Model
history = model.fit(train_generator, epochs=50, verbose=1)

# Evaluate Model on Test Data
test_generator = datagen.flow_from_directory(
    test_dir,  # Update the path
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical')

# Evaluate the model on test data
model.evaluate(test_generator)

Found 10941 images belonging to 8 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


171/171 ━━━━━━━━━━━━━━━━━━━━ 26s 103ms/step - accuracy: 0.3233 - loss: 2.5519
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 32s 80ms/step - accuracy: 0.5441 - loss: 1.2559
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.6176 - loss: 1.0051
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - accuracy: 0.6451 - loss: 0.9470
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.6711 - loss: 0.8728
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.6942 - loss: 0.8289
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.7154 - loss: 0.7886
Epoch 8/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.7194 - loss: 0.8005
Epoch 9/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - accuracy: 0.7355 - loss: 0.7595
Epoch 10/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 21s 86ms/step - accuracy: 0.7390 - loss: 0.7280
Epoch 11/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.7589 - loss: 0.7050
Epoch 12/50
171/171 ━━━━━━━━━━━━━━━━━━━━

[1.4199090003967285, 0.7067833542823792]

 - Epochs 1-50: The model started with a low accuracy (32.33%) and a high loss (2.5519), which is typical for initial epochs.
As training progressed, the accuracy steadily increased, reaching 83.80% by Epoch 50, while the loss decreased to 0.4909.
   This indicates that the model is learning and optimizing its weights to reduce the error.
- Validation accuracy: After training, the model was evaluated on the  
 validation set, achieving an accuracy of 70.68% and a loss of 1.4084.
  Although the validation accuracy is lower than the training accuracy, it suggests the model is generalizing well but might still benefit from further tuning.

To further improve performance, we can:
- Apply data augmentation or more advanced regularization techniques (e.g., Dropout, L2 Regularization).
 - Experiment with a different optimizer or learning rate scheduling to fine-tune the learning process.
- Adjust the network architecture by adding or modifying layers to better capture patterns in the data.
